<a href="https://colab.research.google.com/github/spatank/CIS-522/blob/main/Homework/HW_10_SPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CIS Week 10 Homework

**Instructor:** Lyle Ungar

**Content Creators:** Sanjeevini Ganni, Brittany Shields, Alessandra Rossi

In [1]:
#@markdown What is your Pennkey and pod? (text, not numbers, e.g. bfranklin)
my_pennkey = 'spatank' #@param {type:"string"}
my_pod = 'sublime-newt' #@param ['Select', 'euclidean-wombat', 'sublime-newt', 'buoyant-unicorn', 'lackadaisical-manatee','indelible-stingray','superfluous-lyrebird','discreet-reindeer','quizzical-goldfish','ubiquitous-cheetah','nonchalant-crocodile','fashionable-lemur','spiffy-eagle','electric-emu','quotidian-lion','astute-jellyfish', 'quantum-herring']

# start timing
import time
try:t0;
except NameError: t0 = time.time()

We **strongly** recommend that you keep a separate document offline with your answers, and paste them in when you're ready to submit. Colab may reset and clear your notebook after a period of inactivity.

Go to Runtime -> Change runtime type -> Set Hardware Accelerator to "GPU"

###**Learning Objectives**
Finetune a Transformer model

How might word embedding perpetuate systemic bias?

How might we work with policymakers and companies to reduce the spread of fake news?

## Part I: Finetune GPT-2





GPT2 is a large-scale transformer-based language model developed by OpenAI. It is pre-trained on a large corpus of dataset of 8 million web pages has around 1.5 billion parameters. GPT2 is useful for language generation tasks, that is it predicts the next word, given some text.  

In this excersice we will be finetuning DistillGPT-2, distilled version of GPT2. We want the language model to generate wikipedia style of text. Hence, we will finetune distillGPT on [wikitext-2-raw-v1](https://huggingface.co/datasets/wikitext) dataset scrapped from wikipedia articles.

In [2]:
#@title Install
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.0MB 6.1MB/s 
     |████████████████████████████████| 3.3MB 34.4MB/s 
     |████████████████████████████████| 870kB 40.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=a783084fe47ba44347fe5720eca41a21c7f7e546a47a10f280b5f807201ee0a0
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
     |████████████████████████████████| 194kB 5.4MB/s 
     |████████████████████████████████| 112kB 7.5MB/s 
     |████████████████████████████████| 245kB 6.1MB/s 


In [3]:
#@title Imports
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments

In [4]:
#@title Load Datset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91. Subsequent calls will reuse this data.


Example of Dataset:

In [5]:
datasets["train"][48]

{'text': " Two manga adaptations were produced , following each of the game 's main female protagonists Imca and Riela . They were Senjō no Valkyria 3 : Namo naki Chikai no Hana ( 戦場のヴァルキュリア3 名もなき誓いの花 , lit . Valkyria of the Battlefield 3 : The Flower of the Nameless Oath ) , illustrated by Naoyuki Fujisawa and eventually released in two volumes after being serialized in Dengeki Maoh between 2011 and 2012 ; and Senjō no Valkyria 3 : -Akaki Unmei no Ikusa Otome- ( 戦場のヴァルキュリア3 -赤き運命の戦乙女- , lit . Valkyria of the Battlefield 3 -The Valkyrie of the Crimson Fate ) , illustrated by Mizuki Tsuge and eventually released in a single volume by Kadokawa Shoten in 2012 . \n"}

####Tokenizer

In [6]:
#Define the model Checkpoint
model_checkpoint = "distilgpt2"

#Tokenizer  
#To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. 
#This is all done by the AutoTokenizer class:  
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

#We define a function to call the tokenizer on our texts:
def tokenize_function(examples):
    return tokenizer(examples["text"])

#Apply tokenizer to all the splits in our datasets object and drop the text coolumn
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [7]:
#we need to concatenate all our texts together then split the result in small chunks of a certain block_size
#Define the block size for training
block_size = 128

#preprocessing function that will group our texts
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

###Load pre-trained model

In [8]:
#####Pretrained model
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

###Run Generations before Finetuning

In [9]:
def run_generations(PROMPT_TEXT):
  device = model.device
  input_ids = tokenizer.encode(PROMPT_TEXT, return_tensors="pt").to(device)

  sample_output = model.generate(
      input_ids, 
      do_sample=True, 
      max_length=100, 
      top_p=0.95,
      num_return_sequences=3, 
      early_stopping=True
  )


  for i, output in enumerate(sample_output):
    print("Generation: "+ str(i) +"\n" + 100 * '-')
    print(tokenizer.decode(output, skip_special_tokens=True))
    print()


########
PROMPT_TEXT = 'Fast and the Furious '
run_generations(PROMPT_TEXT)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generation: 0
----------------------------------------------------------------------------------------------------
Fast and the Furious?????? It’s just a good thing for us, our characters.
You know what? I hope that all of you are not being taken by the notion that this is a bad thing? And it’s a good thing for your country. And it’s a great thing for your country because we‖ve got one of the finest stories in the world. If it wasn’t for you, it would probably not have been your

Generation: 1
----------------------------------------------------------------------------------------------------
Fast and the Furious!!!
They have the most amazing and fantastic fans! Their first game is always the one which will bring you to the heart of all the fun!
I think this game must have had some kind of special story for us! Everyone can understand that I am not only a great fan of racing but also a great racer. I love racing, but love racing because it is so fast in that it takes just a fraction of

In [12]:
############## Run Generation for different prompt texts of your choice
PROMPT_TEXT = 'The city of Mumbai'
run_generations(PROMPT_TEXT)


PROMPT_TEXT = 'Historic route 66'
run_generations(PROMPT_TEXT)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generation: 0
----------------------------------------------------------------------------------------------------
The city of Mumbai says it has no plans to remove three buses from the city.




















































































Generation: 1
----------------------------------------------------------------------------------------------------
The city of Mumbai is also gearing up to host three international events on January 7, 2016.


















































































Generation: 2
----------------------------------------------------------------------------------------------------
The city of Mumbai, said the city’s development ministry has been informed about several projects in the next decade.













































































Generation: 0
----------------------------------------------------------------------------------------------------
Historic 

###Fine-Tuning (This will take around 20-25 minutes)

In [13]:
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=2,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,3.757000,3.670282,7.801200,247.527000
2,3.668200,3.654916,7.787200,247.971000


TrainOutput(global_step=4668, training_loss=3.752927543162891, metrics={'train_runtime': 572.6047, 'train_samples_per_second': 8.152, 'total_flos': 2348513500594176.0, 'epoch': 2.0, 'init_mem_cpu_alloc_delta': 384130, 'init_mem_gpu_alloc_delta': 334744576, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 825902, 'train_mem_gpu_alloc_delta': 995278336, 'train_mem_cpu_peaked_delta': 155924730, 'train_mem_gpu_peaked_delta': 1491477504})

Perplexity of Validation Set

In [14]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 38.66


###Run Generations after Finetuning
Run generations for different prompts and compare them with the initial generations.

In [15]:
PROMPT_TEXT = 'Fast and the Furious '
run_generations(PROMPT_TEXT)

############## Run Generation for different prompt texts of your choice
PROMPT_TEXT = 'The city of Mumbai'
run_generations(PROMPT_TEXT)


PROMPT_TEXT = 'Historic route 66'
run_generations(PROMPT_TEXT)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generation: 0
----------------------------------------------------------------------------------------------------
Fast and the Furious 
 = = = Release = = = 
 On November 18, 2012, they had an exclusive exclusive exclusive deal with the production company that brought in their first full production unit. On November 23, 2012, the original version of the title was released on Blu @-@ray in Japan, and the official release on Blu @-@ray in the United States on November 18, 2012 was released exclusively on Blu @-@ray in the United Kingdom. The initial release

Generation: 1
----------------------------------------------------------------------------------------------------
Fast and the Furious 
 = = Release = = 
 In 2008, The Mortal Kombat was announced, and in April 2009, was released as a digital download. At the time, it was the second game of the 2006 reboot, and the second game to be released since 2003. It took the franchise to the next level, and released the game in late September

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generation: 0
----------------------------------------------------------------------------------------------------
The city of Mumbai on the outskirts of Mumbai had been destroyed in the early months of the 19th century and the early 1930s. After the end of World War II, the government of Ravi Shankar Prabhu became part of the State Housing Ministry, and the city of Hyderabad was officially designated a part of Mumbai's Housing Preservation Authority by the 1947 – 1948 Mumbai Urban Municipality Act. 
 The city was originally designated as a part of the National Infrastructure, and in the

Generation: 1
----------------------------------------------------------------------------------------------------
The city of Mumbai in the northwest, its first mosque, built in 1907, serves a similar function to its neighbouring Dhaka Mosque, and is currently being constructed in India. 
 Despite the rise of modernity in India, there are few monuments to Hindu religious practices in the country to d

In [21]:
#@markdown Question 0: How do the generations after finetuning compare to the generations before finetuning. Do you think finetuning acheived its purpose of generating Wikipedia like texts? 
q0 = "Before finetuning the generations resemble news articles. After finetuning, they are closer to Wikipedia-style reports. For instance, the prompt about Mumbai now discusses facets such as geography and history, and not current events like it did pre-finetuning. The same is true for the historic route 66 prompt. None of the content makes a lot of sense, however. " #@param{type:"string"}

try:t0;
except NameError: t0 = time.time()

## Part II: Primer on Gender Bias in Word Embeddings

Read the following primer that describes the societal implications of gender bias in word embeddings: \\
“Man is to Doctor as Woman is to Nurse: The Gender Bias of Word Embeddings: Why we should worry about gender inequality in Natural Language Processing techniques” \\
https://towardsdatascience.com/gender-bias-word-embeddings-76d9806a0e17 \\
*9 min read*, Tommaso Buonocore, 2019


In [22]:
#@markdown Question 1: Identify a compelling quote or example from the article that resonated with you.  How does it emphasize the societal responsibility of data scientists?
q1 = "\"If the model succeeds in retrieving the gender of the candidate, it will be strongly influenced by it during the candidates' selection\" reminds me of the discussion from earlier this semester about protected classes. Gender is one dimension along which the model can learn a bias. Unfortunately, many hidden and explicit biases exist in the data that can be just as harmful.  In our quest to get AI systems to mimic human behavior as closely as possible, we have inadvertently gotten them to mimic and amplify human biases as well. For data scientists, it is key to spend time talking to people that study bias in human behavior and use their expertise to guard against it in tech. " #@param{type:"string"}

try:t1;
except NameError: t1 = time.time()

##Part III: Tech Paper on Gender Bias in Word Embeddings

In the previous article, the author referenced that data scientists are introducing new strategies to reduce biases in word embeddings.  In 2016, researchers from Boston University and Microsoft Research published a paper with significant findings demonstrating the prevalence of gender bias in word embeddings, as well as proposed mitigation measures.  A 5-minute summary of the technical components of the paper can be found [here](https://towardsdatascience.com/tackling-gender-bias-in-word-embeddings-c965f4076a10) and the original paper can be found [here](https://arxiv.org/abs/1607.06520). \

“Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings” \
https://arxiv.org/abs/1607.06520 \
*30 min read*, 2016





In [23]:
#@markdown Question 2: In the “Discussion” section, the authors consider the responsibility of machine language programmers with regards to bias in word embeddings.  Describe their stance.
q2 = "The authors argue that ML systems should at the very least not amplify societal biases. At best, they believe that debiasing the technology can help debias human behavior in the real world." #@param{type:"string"}

#@markdown Question 3: How could word embeddings amplify societal stereotypes?
q3 = "Natural language-based interactions with AI systems are increasingly common. These interactions are not limited to search queries alone but are multi-modal in nature. For instance, a recent Twitter thread highlighted how occupational gender biases transfer between languages during translation (from English to Hungarian) even when the target language has no gendered pronouns (Hungarian). Biases have the potential to create barriers to entry into certain professions for protected classes." #@param{type:"string"}

#@markdown Question 4: How could the authors’ proposed technological interventions address this concern?
q4 = "The authors identify axes that span a gender subspace using equality pairs such as \"he-she\" and \"mother-father\".  The vector for the gender subspace is then removed from gender-specific terms leaving only other meaningful dimensions, a process termed neutralization. Along these other dimensions, vector lengths are equalized so that gendered words do not project more or less along one dimension than another. The collective effect of this approach is the removal of both explicit (neutralization) and implicit (equalization) biases." #@param{type:"string"}

try:t2;
except NameError: t2 = time.time()

##Part IV: Fake News

The following article provides an overview of neural fake news, including potential defenses against it as well as suggestions for further research.

“An Exhaustive Guide to Detecting and Fighting Neural Fake News using NLP” \
•	https://www.analyticsvidhya.com/blog/2019/12/detect-fight-neural-fake-news-nlp/ \
•	*16-minute read*, Analytics Vidhya, 2019

In 2019, Jack Clark, the Policy Director at OpenAI, gave testimony to the US House Intelligence Committee.  You can either read his written 10-page testimony here or watch his live testimony here (jump to minute 11:00 to watch his 5 minute testimony).

Written Testimony of Jack Clark, Policy Director, OpenAI \
•	https://intelligence.house.gov/uploadedfiles/clark_deepfakes_sfr.pdf \
•	or \
•	https://www.c-span.org/video/?461679-1/house-intelligence-committee-hearing-deepfake-videos \
*Jump to minute 11:00* \
•	House Permanent Select Committee on Intelligence, 2019 \


In [24]:
#@markdown Question 5: Jack Clark enumerated several ideas for interventions.  Discuss one of the specific ideas he suggested.
q5 = "Jack Clark discussed a potential threat intelligence sharing mechanism between large corporations specifically meant to target fake news and malicious deep fakes. " #@param{type:"string"}

#@markdown Question 6: How might technologists work with policymakers and companies to prevent the spread of fake news?
q6 = "The problem of dealing with fake news is a fundamentally partisan issue. In my view, the solution in these circumstances is bipartisanship in the form of a nationally mandated FTC-style organization for AI ethics and policy." #@param{type:"string"}

#@markdown Question 7: •	OpenAI is not making GPT-3 publicly available, claiming that they are concerned that it might be used for doing bad. Do you think that was a good decision? Why?
q7 = "I think generally being cautious about releasing state-of-the-art technology is reasonable. At the same time, it is also important for OpenAI to make their models available to good faith actors such as researchers and non-profits. A lot of critical manufacturing equipment (or even weaponry) has GPS-based software that prevents it from being used by non-pre-approved entities. The same is now possible with digital technology. It can be selectively released and then tracked through its deployed life cycle." #@param{type:"string"}

try:t3;
except NameError: t3 = time.time()

## PART V: Know your Pod

With two other members of your pod, do the following. Have each of them recommend you their favorite song. Then, listen to those songs and write down your thoughts for each. (~100 words each)

In [ ]:
know_a_pod_1 = "" #@param{type:"string"}
know_a_pod_2 = "" #@param{type:"string"}

try:t4;
except NameError: t4 = time.time()

#Submission

Once you're done, click on 'Share' and add the link to the box below.

In [25]:
link = 'https://colab.research.google.com/drive/1nWOr8AJFzu-wkylzb18ABGYs2-T9pzj9?usp=sharing' #@param {type:"string"}

In [26]:
import time
import numpy as np
import urllib.parse
from IPython import display
from IPython.display import IFrame


t5 = time.time()

#@markdown #Run Cell to Show Airtable Form
#@markdown ##**Confirm your answers and then click "Submit"**


def prefill_form(src, fields: dict):
  '''
  src: the original src url to embed the form
  fields: a dictionary of field:value pairs,
  e.g. {"pennkey": my_pennkey, "location": my_location}
  '''
  prefill_fields = {}
  for key in fields:
      new_key = 'prefill_' + key
      prefill_fields[new_key] = fields[key]
  prefills = urllib.parse.urlencode(prefill_fields)
  src = src + prefills
  return src



#autofill fields if they are not present
#a missing pennkey and pod will result in an Airtable warning
#which is easily fixed user-side.
try: my_pennkey;
except NameError: my_pennkey = ""

try: my_pod;
except NameError: my_pod = "Select"

try: q0;
except NameError: q0 = ""

try: q1;
except NameError: q1 = ""

try: q2;
except NameError: q2 = ""

try: q3;
except NameError: q3 = ""

try: q4;
except NameError: q4 = ""

try: q5;
except NameError: q5 = ""

try: q6;
except NameError: q6 = ""

try: q7;
except NameError: q7 = ""

try: know_a_pod_1;
except NameError: know_a_pod_1 = ""

try: know_a_pod_2;
except NameError: know_a_pod_2 = ""

try: link;
except NameError: link = ""

fields = {"pennkey": my_pennkey,
          "pod": my_pod,
          "q0": q0,
          "q1": q1,
          "q2": q2,
          "q3": q3,
          "q4": q4,
          "q5": q5,
          "q6": q6,
          "q7": q7,
          "know_a_pod_1": know_a_pod_1,
          "know_a_pod_2": know_a_pod_2,
          "link": link}

src = "https://airtable.com/embed/shrgundiXUEJFVzwd?"


#now instead of the original source url, we do: src = prefill_form(src, fields)
display.display(IFrame(src = prefill_form(src, fields), width = 800, height = 400))